# Benzinga-Nachrichten-Verarbeitung

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cwd = "/content/drive/MyDrive/NewsTrading/trading_bot"
%cd /content/drive/MyDrive/NewsTrading/trading_bot

/content/drive/MyDrive/NewsTrading/trading_bot


In [3]:
!sudo apt update
!sudo apt install maven;

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,002 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [518 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,081 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,226 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:12 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelea

In [4]:
%%capture
!pip install html2text
!pip install datefinder
!pip install -U dask[complete]
!pip install nltk
!pip install dateparser
!pip install sutime;

In [5]:
!mvn dependency:copy-dependencies -DoutputDirectory=./jars -f $(python3 -c 'import importlib; import pathlib; print(pathlib.Path(importlib.util.find_spec("sutime").origin).parent / "pom.xml")');


[INFO] Scanning for projects...
Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-compiler-plugin/3.8.1/maven-compiler-plugin-3.8.1.pom (12 kB at 36 kB/s)
Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-plugins/33/maven-plugins-33.pom (11 kB at 172 kB/s)
Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-parent/33/maven-parent-33.pom (44 kB at 1.3 MB/s)
Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/apache/21/apache-21.pom (17 kB at 1.1 MB/s)
Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-compiler-plugin/3.8.1/maven-compiler-plugin-3.8.1.jar (62 kB at 2.7 MB/s)
Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-jar-plugin/3.2.0/maven-jar-plugin-3.2.0.pom (7.3 kB at 726 kB/s)
Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/p

In [6]:
%load_ext autoreload
%autoreload 2
import dask.dataframe as dd
import dask
import pandas as pd
from dask.distributed import Client
from src.preprocessing.news_parser import filter_body, time, body_formatter, get_company_abbreviation
import re
import plotly.express as px
import nltk
nltk.download('punkt')
import yfinance as yf

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [7]:
# client = Client(memory_limit='25GB', processes=False,
#                 n_workers=2, threads_per_worker=1)
# client

## Grobes HTML-Parsing
Als erstes müssen wir die HTML-Dokumente zu normalem Text umwandeln, ansonsten sind die Text-Zellen zu groß und führen zu Problemen mit PyArrow/Dask.

In [ ]:
dask.config.set(scheduler="threads")

In [ ]:
input_dir = "data/raw_bzg/"
output_dir = 'data/unraw1_bzg/'

In [ ]:
# for year in range(2019, 2020):
#     print(year)
#     df = pd.read_parquet(f"{input_dir}story_df_raw_{year}.parquet")
#     df = dd.from_pandas(df, npartitions=12)
#     df["html_body"] = df["html_body"].apply(body_formatter, meta=pd.Series(dtype="str"))
#     df = df.rename(columns={"html_body":"body"})
#     name_function = lambda x: f"data-{year}-{x}.parquet"
#     df.to_parquet(output_dir, name_function=name_function)

## Neu-Partitionierug
Sodass alle Partitionen etwa die gleiche Größe haben.

In [ ]:
input_dir = 'data/unraw1_bzg/'
output_dir = 'data/unraw2_bzg/'

# ddf = dd.read_parquet(input_dir+"*.parquet")
# ddf2 = ddf.repartition(npartitions=50)
# name_function = lambda x: f"data-{x}.parquet"
# ddf2.to_parquet(output_dir, name_function=name_function)

## Author-Inferenz

Ein bisschen die Daten säubern...

In [ ]:
input_dir = cwd+'/data/unraw2_bzg/'
output_dir = cwd+'/data/unraw3_bzg/'

In [ ]:
ddf = dd.read_parquet(input_dir+"*.parquet")

In [ ]:
# Remove rows for which noo stock ticker is recorded
ddf = ddf[ddf.stocks != '']

In [ ]:
# Convert `channels`  datatype from string to list
ddf["channels"] = ddf["channels"].apply(eval, meta=pd.Series(dtype='object'))

Untersuche als nächstes die Behauptung, dass **PRNewswire** und **Businesswire** den gesamten Markt für Pressemeldungen in den USA kontrollieren. Wenn dem so ist, und sie nicht noch weitere, unwichtige Meldungen veröffentlichen, dann können wir einfach die Newsartikel nach diesen Autoren filtern und uns viel Arbeit ersparen.

In [ ]:
dask.config.set(scheduler="processes")
ddf["inferred_author"] = None

def infer_author(body):
  for author in ["PRNewswire", "Globe Newswire", "Business Wire", "ACCESSWIRE"]:
    if re.search(author, body, re.IGNORECASE) is not None:
      return author
  return None

ddf["inferred_author"] = ddf.body.apply(infer_author, meta=pd.Series(dtype="string"))

In [ ]:
# value_counts for authors
auhtor_value_counts = pd.concat([ddf.author.value_counts().head(10), ddf.inferred_author.value_counts().head(10)], axis=1)

In [ ]:
auhtor_value_counts

,author,inferred_author
Benzinga,1061214,NaN
PRNewswire,305720,587242.0
Globe Newswire,293466,475171.0
Business Wire,268561,293052.0
Newsfile,70877,NaN
ACCESSWIRE,62615,81054.0
"AB Digital, Inc.",9936,NaN
WebWire,6404,NaN
PRWeb,2617,NaN
News Direct,2080,NaN


In [ ]:
auhtor_value_counts.sum().diff()

author                  NaN
inferred_author   -646971.0
dtype: float64

Ungefähr 650k Nachrichten werden ausgelassen, wenn nur die vier Hauptvertreiber von Pressemeldungen berücksichtigt werden.

In [ ]:
ddf = ddf[~ddf.inferred_author.isna()]

In [ ]:
ddf["inferred_author"] = ddf["inferred_author"].astype("string")

In [ ]:
ddf["channels"] = ddf.channels.apply(lambda x: str(x), meta=pd.Series(dtype="string"))

In [ ]:
ddf.inferred_author.value_counts().compute()

PRNewswire        587242
Globe Newswire    475171
Business Wire     293052
ACCESSWIRE         81054
Name: inferred_author, dtype: Int64

In [ ]:
ddf.inferred_author.value_counts().sum().compute()

1436519

In [ ]:
name_function = lambda x: f"data-{x}.parquet"
ddf.to_parquet(output_dir, name_function=name_function)

In [ ]:
# Contains 100k rows
earnings_ddf = ddf[ddf.channels.apply(lambda x: "Earnings" in x, meta=pd.Series(dtype=bool))]

In [ ]:
# value counts for authors of earnings reports (contrast to value counts of all news articles)
earnings_ddf.inferred_author.value_counts().head(10)

Globe Newswire    44589
PRNewswire        31440
ACCESSWIRE        16434
Name: inferred_author, dtype: Int64

Hier sehen wir, dass es keine einzige Pressemeldung von **Business Wire** gibt, die mit *Earnings* gekennzeichnet sind. Trotzdem gibt es relevante *Earnings* reports von Business Wire. Dies habe ich kurz verifiziert...

## Vorgehen

Benötigt:
---------
- data/tickers.pkl (alt) \\
- data_shared/corporation_endings.txt \\
- input_dir = data/unraw3_bzg/ \\
- output_dir = data/unraw4_bzg/

Produziert:
-----------
- data_shared/all_ticker_name_mapper.parquet

--------------

Wie viele Nachrichten bleiben, wenn wir auf relevante Ticker filtern? Wir wollen nicht(!) - so ist es momentan - auf die momentane Russell 3k-Zusammensetzung filtern, denn wir wollen auch ungelistete bzw. ehemalige Russell-Aktien beachten.


**1. Full-Name-Discovery:**

Herausfinden des vollen Namens des Unternehmens für jeden Ticker, damit 1. der Text richtig geparst werden kann und 2. damit wir einen Anhaltspunkt für das Ticker-Grouping haben.


**2. Ticker-Filtering:**

Alle Ticker herausfiltern, die wir nicht brauchen. Wenn wir aber ein großes Aktienuniversum (mit inzwischen ungelisteten Aktien) benutzen, werden wir fast alle Nachrichten behalten können. Allerdings lassen sich so Fehlerhafte Nachrichten/Ticker etc. herausfiltern.


**3. Ticker-Grouping:**

Was machen wir, wenn wir mehrerer Aktiengattungen für ein Unternehmen haben? Z.B. Vorzugs- und Stammaktien. Wir können i.A. die Stammaktie nehmen, da diese normalerweise ein höheres Handelsvolumen aufweist. D.h. wir bilden alle Ticker der Benzinga-Nachrichten auf den Ticker der Stammaktie ab.


**4. Firmennamen-Nachrichtenkörper-Verifikation:**

Da Ticker wiederverwendet werden können bzw. sich verändern können wollen wir sicherstellen, dass der Unternehmensname im Nachrichtenkörper vorkommt! Bzw. generell ist das eine gute Datensäuberungs-Maßnahme.

In [ ]:
# Get company name by ticker (longName is always equal to shortName in yf...)
# This takes a long time, because of the api calls to yf (15min)
def yahoo_get_wrapper(x):
  try:
    return yf.Ticker(x).info.get("longName")
  except:
    return None

In [ ]:
input_dir = cwd+'/data/unraw3_bzg/'
output_dir = cwd+'/data/unraw4_bzg/'
ddf = dd.read_parquet(input_dir)

In [ ]:
all_tickers = ddf.stocks.unique().compute()

### Full-Name-Discovery

In [ ]:
company_names = all_tickers.apply(lambda x: yahoo_get_wrapper(x))

In [ ]:
all_mapper = pd.concat([all_tickers, company_names], axis=1)

In [ ]:
all_mapper.columns = ["ticker", "company_name"]

In [ ]:
all_mapper = all_mapper.dropna() # This drops like half of the tickers.

In [ ]:
all_mapper[all_mapper.company_name.apply(lambda x: "Alphabet" in x)]

,ticker,company_name
653,GOOG,Alphabet Inc.
10617,GOOGL,Alphabet Inc.


In [ ]:
print(len(all_mapper))
print(len(all_mapper.company_name.unique()))

11383
11072


In [ ]:
vcs = all_mapper.company_name.value_counts()
vcs = vcs[vcs >= 2]

In [ ]:
# Ticker-Grouping
all_mapper[all_mapper.company_name.isin(vcs.index)].sort_values("company_name")

,ticker,company_name
21729,VCXA,10X Capital Venture Acquisition Corp. II
19821,VCXAU,10X Capital Venture Acquisition Corp. II
20368,ADER,26 Capital Acquisition Corp.
18164,ADERU,26 Capital Acquisition Corp.
18032,VIIAU,7GC & Co. Holdings Inc.
...,...,...
15333,ICCT,iCoreConnect Inc.
21288,IONR,ioneer Ltd
20748,GSCCF,ioneer Ltd
18265,OSSFF,Össur hf.


Es ist nicht leicht zu sagen, welchen von den Tickern wir bevorzugen sollten. Abgleichen mit den Aktientickern des Kursdatensatzes notwendig, um zu sehen, ob überhaupt nur ein Ticker übereinstimmt. Wenn es für beide Ticker eine Kurszeitreihe gibt, dann sollten wir die nehmen, die ein höheres historisches Volumen hat. Dies ist allerdings etwas, was wir später machen und nicht jetzt. Hier wollen wir zunächst nur die Nachrichten verarbeiten, weswegen wir nur die NaN-Unternehmen rausnehmen und den Rest - *ohne Ticker-Filtering* - weiterverarbeiten.

In [ ]:
mapper = all_mapper

In [ ]:
mapper.columns = ["ticker", "company_names"]
mapper = mapper[mapper.isna().sum(axis=1) == 0]

In [ ]:
mapper = mapper.set_index("ticker")

In [ ]:
company_endings = pd.read_table("data_shared/corporation_endings.txt").iloc[:, 0]
# Apply get_company_abbreviation twice in order to get rid of Enterprise, Ltd.
# Otherwise , Ltd. remains.
mapper["short_name"] = mapper.company_names.apply(lambda x: get_company_abbreviation(x, company_endings=company_endings))

In [ ]:
print(mapper.short_name.isna().sum()) # 2037 stocks for which we don't have an ending to abbreviate
# mapper.loc[:, "short_name"] = mapper.short_name.fillna(mapper.company_names)
mapper = mapper.applymap(lambda x: x.strip(" "))

0


In [ ]:
mapper.to_parquet(cwd + "/data_shared/ticker_name_mapper.parquet")

In [ ]:
mapper = pd.read_parquet(cwd + "/data_shared/ticker_name_mapper.parquet")

In [ ]:
filt_ddf = ddf[ddf.stocks.isin(mapper.index.to_list())]

In [ ]:
ddf.shape[0].compute()

1436519

In [ ]:
filt_ddf.shape[0].compute()

1017673

Es verbleiben circa 1 Mio. Nachrichten, für die wir den Ticker zu einem FIrmennamen auflösen können.

In [ ]:
filt_ddf = filt_ddf.set_index("time")

In [ ]:
monthly_news_counts = filt_ddf.stocks.resample("MS").count()

In [ ]:
# px.line(monthly_news_counts.compute(), title="# of articles per month")

In [ ]:
dask.config.set(scheduler="processes")

Diese Nachrichten können wir nun wirklich parsen, und danach ordentlich kategorisieren.

In [ ]:
ddf = filt_ddf
ddf = ddf.drop(columns=["author"]).rename(columns={"inferred_author":"author"})

In [ ]:
# TODO: Kann effizienter werden mit GroupBy-Operation
ddf["company_name"] = ddf.stocks.apply(lambda x: mapper.company_names.loc[x], meta=pd.Series(dtype="string"))
ddf["short_name"] = ddf.stocks.apply(lambda x: mapper.short_name.loc[x], meta=pd.Series(dtype="string"))

In [ ]:
name_function = lambda x: f"data-{x}.parquet"
ddf.to_parquet(cwd+'/data/latest/', name_function=name_function)

In [18]:
ddf = dd.read_parquet(cwd+'/data/latest/')

### Firmennamen-Nachrichtenkörper-Verifikation

In [19]:
mask = ddf.apply(lambda x: bool(re.search(x["short_name"], x["body"].replace("\n", " "), re.IGNORECASE)) or \
                 bool(re.search(x["short_name"], x.title, re.IGNORECASE)),
                 axis=1,
                 meta=pd.Series(dtype=bool))

In [ ]:
# Around 11k stocks before `filtering`
# len(ddf.stocks.unique())

# Around 10k stocks after `filtering`
# len(ddf[mask].stocks.unique())

11383

In [ ]:
ddf[mask].shape[0].compute()

897403

In [ ]:
# Case 1: Firmenname wird in abgekürzter Version benutzt, die wir nicht methodisch rekonstruieren können
#   Z.B.: IBM -> International Business Machines,
#   oder UPS -> United Parcel Service
#   oder GE -> General Electric
# Case 2: Ticker wurde recycled und zeigt inzwischen auf eine andere Firma.
# Will hierzu nicht auch noch Daten kaufen, deswegen werden hier leider einige Nachrichten verworfen werden.

ddf[~mask].shape[0].compute() # ~120k Nachrichten mit `FALSCHEM` Firmennamen -> Textinhalt ableichen mit Firmennamen des Kursdaten-Datensatzes.

120270

In [ ]:
ddf[~mask].stocks.value_counts().compute().head(15)

IBM     1571
UPS     1438
GE      1397
FMCC    1382
S       1285
APD      953
CI       914
IPG      830
ABT      717
RENT     703
AWK      701
HLT      648
ERIC     639
EMC      622
UHAL     614
Name: stocks, dtype: Int64

In [20]:
# Filter for high conviction entries
ddf = ddf[mask]

In [21]:
# Repartition
ddf = ddf.repartition(npartitions=30)

## Nachrichten-Parsing


In [ ]:
dask.config.set(scheduler="processes")

In [ ]:
sample_partition = ddf.get_partition(20)
x = sample_partition.head(5).iloc[0]
x.body

In [ ]:
filter_body(x.body, x.stocks, x.author, x.name, x.company_name, x.short_name) # x.name == time

Anwenden der filter_body-Funktion auf alle Reihen:

In [28]:
ddf["parsed_body"] = ddf.apply(lambda x: filter_body(x.body,
                                                      x.stocks,
                                                      x.author,
                                                      x.name,
                                                      x.company_name,
                                                      x.short_name),
                                axis=1,
                                meta=pd.Series(dtype="string"))

In [29]:
ddf = ddf.drop(columns=["body"])

In [ ]:
output_dir = 'data/parsed_bzg/'
name_function = lambda x: f"data-{x}.parquet"
ddf.to_parquet(output_dir, name_function=name_function)

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
SUTime didn't approve as a date: at 5:00 p.m. EST / 2:00 p.m. PST on Tuesday, February 5, 2013
SUTime didn't approve as a date: February 19, 2013 at 5:00 p.m. EST / 2:00 p.m. PST
SUTime didn't approve as a date: 2,250,000
SUTime didn't approve as a date: Thursday August 4, 2011, at 10:00 a.m. Central / 11:00 a.m. East
SUTime didn't approve as a date: 8:30 AM ET on October 28, 2014
SUTime didn't approve as a date: 2,700,000
SUTime didn't approve as a date: 12:30 p.m. ET on May 7, 2013
SUTime didn't approve as a date: April 4-12, 2011
SUTime didn't approve as a date: 9,998,107
SUTime didn't approve as a date: on Tuesday, April 24, 2012, at 8:00 a.m. PDT
SUTime didn't approve as a date: at 2:30 pm ET on December 10, 2013, at
SUTime didn't approve as a date: on Thursday, December 5, 2013, at 11:30 a.m. MST
SUTime didn't approve as a date: 8,846,034
SUTime didn't approve as a date: on Tuesday, July 1, 2014, at 5:00 p.m. ET
S

In [ ]:
import requests
import time

while True:
    try:
        requests.get('https://www.google.com')
        print("Kept alive.")
    except:
        print("Failed to keep alive.")
    time.sleep(600)